In [3]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum


class IssueCategory(str, Enum):
    """Категории обращений"""
    ORDER_STATUS = "order_status"  # Статус заказа
    INCOMPLETE_ORDER = "incomplete_order"  # Неполнота заказа
    DAMAGED_ITEM = "damaged_item"  # Повреждение товара
    OTHER = "other"  # Другое


class Confidence(str, Enum):
    """Уровень уверенности в категории"""
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


class Issue(BaseModel):
    """Отдельный вопрос/проблема из отзыва"""
    category: IssueCategory = Field(
        description="Категория обращения"
    )
    
    confidence: Confidence = Field(
        description="Уверенность в категории: high, medium, low"
    )
    
    order_number: Optional[str] = Field(
        default=None,
        description="Номер заказа, если упомянут"
    )
    
    sentiment: str = Field(
        description="Тональность: positive, negative, neutral"
    )


class ReviewRouting(BaseModel):
    """Результат роутинга отзыва"""
    issues: List[Issue] = Field(
        description="Список выявленных вопросов/проблем",
        min_items=1
    )
    
    overall_sentiment: str = Field(
        description="Общая тональность отзыва: positive, negative, neutral, mixed"
    )



In [10]:
import json
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum


class IssueCategory(str, Enum):
    """Категории обращений"""
    ORDER_STATUS = "order_status"
    INCOMPLETE_ORDER = "incomplete_order"
    DAMAGED_ITEM = "damaged_item"
    OTHER = "other"


class Confidence(str, Enum):
    """Уровень уверенности в категории"""
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


class ComplexityLevel(str, Enum):
    """Сложность вопроса/проблемы"""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"


class UrgencyLevel(str, Enum):
    """Срочность обращения"""
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"


class Issue(BaseModel):
    """Отдельный вопрос/проблема из отзыва"""
    category: IssueCategory = Field(description="Категория обращения")
    confidence: Confidence = Field(description="Уверенность в категории")
    order_number: Optional[str] = Field(default=None, description="Номер заказа, если упомянут")
    sentiment: str = Field(description="Тональность: positive, negative, neutral")
    complexity: ComplexityLevel = Field(description="Сложность вопроса: low, medium, high")
    urgency: UrgencyLevel = Field(description="Срочность вопроса: low, medium, high")


class ReviewRouting(BaseModel):
    """Результат роутинга отзыва"""
    issues: List[Issue] = Field(description="Список выявленных вопросов/проблем", min_items=1)
    overall_sentiment: str = Field(description="Общая тональность отзыва: positive, negative, neutral, mixed")


def route_review(review_text: str, api_key: str, model: str = "openai/gpt-4o") -> ReviewRouting:
    """
    Роутинг отзыва с schema-guided reasoning
    """
    client = OpenAI(
        api_key=api_key,
        base_url="https://openrouter.ai/api/v1"
    )
    
    completion = client.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": """Ты - система анализа отзывов на магазин. 

Категории проблем:
- order_status: вопросы о статусе заказа, где заказ, когда придет
- incomplete_order: в заказе чего-то не хватает
- damaged_item: товар поврежден, сломан, не работает
- other: всё остальное

Для каждого выявленного вопроса или проблемы оцени:
- complexity: сложность решения (low — решается просто/по шаблону, medium — требует внимания или координации, high — требует индивидуальной работы, вручного решения, нестандартного подхода)
- urgency: срочность (high — требует немедленного ответа или срочного вмешательства, medium — желательно быстрое решение, low — можно обработать в обычном порядке)

Если в отзыве несколько разных вопросов/проблем - создай отдельный Issue для каждого.
"""
            },
            {
                "role": "user",
                "content": review_text
            }
        ],
        response_format=ReviewRouting,
        temperature=0.1
    )
    
    return completion.choices[0].message.parsed



In [11]:
test_reviews = [
    # 1. ORDER_STATUS - простой случай
    {
        "text": "Здравствуйте! Заказ №45678 оформил 5 дней назад, но до сих пор не пришел. Где мой заказ? Когда будет доставка?",
        "expected": ["order_status"]
    },
    
    # 2. INCOMPLETE_ORDER - явный случай
    {
        "text": "Получил заказ 12345, но вместо трех футболок пришло только две! Одной не хватает. Что делать?",
        "expected": ["incomplete_order"]
    },
    
    # 3. DAMAGED_ITEM - повреждение товара
    {
        "text": "Ужас! Заказ №99001 пришел, но телефон разбитый! Экран весь в трещинах, упаковка помята. Требую возврат денег!",
        "expected": ["damaged_item"]
    },
    
    # 4. OTHER - благодарность
    {
        "text": "Спасибо большое! Все пришло быстро, качество отличное. Буду заказывать еще. Рекомендую магазин!",
        "expected": ["other"]
    },
    
    # 5. Множественные проблемы - incomplete + damaged
    {
        "text": "Заказ 55555 наконец пришел, но проблем куча: не хватает одной пары носков из заказа, а куртка вся в пятнах каких-то! Это что за сервис?",
        "expected": ["incomplete_order", "damaged_item"]
    },
    
    # 6. Множественные проблемы - order_status + incomplete
    {
        "text": "Жду заказ 77777 уже 2 недели, трек не обновляется. Плюс звонили, сказали что одного товара нет в наличии - почему сразу не предупредили?",
        "expected": ["order_status", "incomplete_order"]
    },
    
    # 7. Множественные проблемы - все три категории
    {
        "text": "Кошмар с заказом 88888! Во-первых, ждал месяц, трек вообще не отслеживался. Во-вторых, когда наконец пришло - одних наушников нет! В-третьих, ноутбук поцарапан весь, коробка рваная!",
        "expected": ["order_status", "incomplete_order", "damaged_item"]
    },
    
    # 8. OTHER - общий вопрос (не про конкретный заказ)
    {
        "text": "Подскажите, у вас есть доставка в Казахстан? И какие способы оплаты доступны? Хочу заказать, но не уверен.",
        "expected": ["other"]
    },
    
    # 9. Неоднозначный случай - может быть incomplete или damaged (low confidence)
    {
        "text": "Заказ 33333 получил, но что-то не то с ботинками... Один ботинок какой-то странный, не знаю, бракованный или просто не тот размер положили?",
        "expected": ["damaged_item", "other"]  # может быть разная интерпретация
    },
    
    # 10. Смешанный отзыв - positive + order_status
    {
        "text": "В целом магазин хороший, товары качественные! Но вот с заказом 11111 какая-то задержка, уже неделя прошла. Когда отправите?",
        "expected": ["order_status"]
    },
]
api_key = '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'
# Функция для тестирования
def test_routing():
    for i, review in enumerate(test_reviews, 1):
        print(f"\n{'='*80}")
        print(f"ТЕСТ {i}")
        print(f"{'='*80}")
        print(f"Отзыв: {review['text']}")
        print(f"\nОжидаемые категории: {review['expected']}")
        
        try:
            result = route_review(review['text'], api_key)
            print(f"\n✅ РЕЗУЛЬТАТ:")
            print(json.dumps(result.model_dump(), indent=2, ensure_ascii=False))
            
            # Проверка категорий
            found_categories = [issue.category for issue in result.issues]
            print(f"\nНайденные категории: {found_categories}")
            
        except Exception as e:
            print(f"\n❌ ОШИБКА: {e}")

# Запуск тестов
if __name__ == "__main__":
    test_routing()


ТЕСТ 1
Отзыв: Здравствуйте! Заказ №45678 оформил 5 дней назад, но до сих пор не пришел. Где мой заказ? Когда будет доставка?

Ожидаемые категории: ['order_status']

✅ РЕЗУЛЬТАТ:
{
  "issues": [
    {
      "category": "order_status",
      "confidence": "high",
      "order_number": "45678",
      "sentiment": "negative",
      "complexity": "low",
      "urgency": "high"
    }
  ],
  "overall_sentiment": "negative"
}

Найденные категории: [<IssueCategory.ORDER_STATUS: 'order_status'>]

ТЕСТ 2
Отзыв: Получил заказ 12345, но вместо трех футболок пришло только две! Одной не хватает. Что делать?

Ожидаемые категории: ['incomplete_order']

✅ РЕЗУЛЬТАТ:
{
  "issues": [
    {
      "category": "incomplete_order",
      "confidence": "high",
      "order_number": "12345",
      "sentiment": "negative",
      "complexity": "medium",
      "urgency": "medium"
    }
  ],
  "overall_sentiment": "negative"
}

Найденные категории: [<IssueCategory.INCOMPLETE_ORDER: 'incomplete_order'>]

ТЕСТ 3
Отзы

In [12]:
comment = 'Кошмар с заказом 88888! Во-первых, ждал месяц, трек вообще не отслеживался. Когда наконец пришло - одних наушников нет! А, ноутбук поцарапан весь, коробка рваная! Продавец нахамил сказала не наши проблемы!!!'
result = route_review(comment, api_key)
print(f"\n✅ РЕЗУЛЬТАТ:")
print(json.dumps(result.model_dump(), indent=2, ensure_ascii=False))

# Проверка категорий
found_categories = [issue.category for issue in result.issues]
print(f"\nНайденные категории: {found_categories}")


✅ РЕЗУЛЬТАТ:
{
  "issues": [
    {
      "category": "order_status",
      "confidence": "high",
      "order_number": "88888",
      "sentiment": "negative",
      "complexity": "medium",
      "urgency": "medium"
    },
    {
      "category": "incomplete_order",
      "confidence": "high",
      "order_number": "88888",
      "sentiment": "negative",
      "complexity": "medium",
      "urgency": "medium"
    },
    {
      "category": "damaged_item",
      "confidence": "high",
      "order_number": "88888",
      "sentiment": "negative",
      "complexity": "high",
      "urgency": "high"
    },
    {
      "category": "other",
      "confidence": "medium",
      "order_number": "88888",
      "sentiment": "negative",
      "complexity": "medium",
      "urgency": "medium"
    }
  ],
  "overall_sentiment": "negative"
}

Найденные категории: [<IssueCategory.ORDER_STATUS: 'order_status'>, <IssueCategory.INCOMPLETE_ORDER: 'incomplete_order'>, <IssueCategory.DAMAGED_ITEM: 'damaged_it